In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
base_dir = '/content/drive/MyDrive/Real and Fake Face Detection Dataset'
real_dir = os.path.join(base_dir, 'Real')
fake_dir = os.path.join(base_dir, 'Fake')

# ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training and validation generators
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 1633 images belonging to 2 classes.
Found 408 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
def create_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
input_shape = (150, 150, 3)
cnn_model = create_cnn_model(input_shape)

history_cnn = cnn_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10
52/52 [==============================] - 378s 7s/step - loss: 0.7025 - accuracy: 0.5199 - val_loss: 0.6921 - val_accuracy: 0.5466
Epoch 2/10
52/52 [==============================] - 99s 2s/step - loss: 0.6718 - accuracy: 0.6007 - val_loss: 0.7286 - val_accuracy: 0.5441
Epoch 3/10
26/52 [==============>...............] - ETA: 42s - loss: 0.6525 - accuracy: 0.6117

In [ ]:
# Save the CNN model
cnn_model.save('/content/drive/My Drive/Real_and_Fake_Faces/cnn_model.h5')

In [ ]:
from tensorflow.keras.applications import VGG16

# Define the transfer learning model
def create_transfer_learning_model(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
input_shape = (150, 150, 3)
tl_model = create_transfer_learning_model(input_shape)

history_tl = tl_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


In [ ]:
# Save the Transfer Learning model
tl_model.save('/content/drive/My Drive/Real_and_Fake_Faces/tl_model.h5')

In [ ]:
# Evaluate CNN model
cnn_model.evaluate(validation_generator)

# Evaluate Transfer Learning model
tl_model.evaluate(validation_generator)

# Plotting for CNN model
plt.figure(figsize=(12, 8))

# Plot CNN model accuracy
plt.subplot(2, 2, 1)
plt.plot(history_cnn.history['accuracy'])
plt.plot(history_cnn.history['val_accuracy'])
plt.title('CNN Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot CNN model loss
plt.subplot(2, 2, 2)
plt.plot(history_cnn.history['loss'])
plt.plot(history_cnn.history['val_loss'])
plt.title('CNN Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plotting for Transfer Learning model
# Assuming `history_tl` contains the history for Transfer Learning model

# Plot Transfer Learning model accuracy
plt.subplot(2, 2, 3)
plt.plot(history_tl.history['accuracy'])
plt.plot(history_tl.history['val_accuracy'])
plt.title('Transfer Learning Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot Transfer Learning model loss
plt.subplot(2, 2, 4)
plt.plot(history_tl.history['loss'])
plt.plot(history_tl.history['val_loss'])
plt.title('Transfer Learning Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
# Evaluate CNN model
cnn_val_loss, cnn_val_acc = cnn_model.evaluate(validation_generator)
print(f"CNN Model - Validation Loss: {cnn_val_loss}, Validation Accuracy: {cnn_val_acc}")

# Evaluate Transfer Learning model
tl_val_loss, tl_val_acc = tl_model.evaluate(validation_generator)
print(f"Transfer Learning Model - Validation Loss: {tl_val_loss}, Validation Accuracy: {tl_val_acc}")


In [ ]:
# Function to preprocess image
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image
    return img_array

# Function to predict using the model
def predict_image(model, image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)

    if prediction[0] > 0.5:
        return "Fake"
    else:
        return "Real"


In [ ]:
# Path to the test image
test_image_path = '/content/drive/MyDrive/Real and Fake Face Detection Dataset/Fake/mid_80_1111.jpg'

# Predict using the CNN model
result_cnn = predict_image(cnn_model, test_image_path)
print(f"CNN Model Prediction: {result_cnn}")

# Predict using the Transfer Learning model
result_tl = predict_image(tl_model, test_image_path)
print(f"Transfer Learning Model Prediction: {result_tl}")